<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/2025_06_19(2)%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 4.7 MB/s eta 0:00:00


In [5]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [6]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [14]:
# ✅ 라이브러리 CatBoost 단독 모델 + F1 기반 Threshold 최적화 전체 코드
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

# ✅ 데이터 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 (라벨 인코딩 + 결측치 처리)
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ CatBoost 단독 모델 정의
cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    eval_metric='Logloss',
    loss_function='Logloss',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50,
    task_type='CPU'  # GPU 환경이면 'GPU'
)

# ✅ OOF 예측값으로 Threshold 최적화
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print(f"\n🌀 Fold {fold+1}")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
    oof_preds[val_idx] = cat_model.predict_proba(X_val)[:, 1]
    test_preds += cat_model.predict_proba(X_test)[:, 1] / cv.n_splits

# ✅ Threshold 튜닝
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"\n✅ Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 예측 및 저장
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_catboost_thresh_opt.csv", index=False)
print("✅ CatBoost 단독 + Threshold 최적화 결과 저장 완료!")



🌀 Fold 1
0:	learn: 0.6679242	test: 0.6680062	best: 0.6680062 (0)	total: 77.4ms	remaining: 1m 17s
100:	learn: 0.3089050	test: 0.3110908	best: 0.3110908 (100)	total: 7.41s	remaining: 1m 5s
200:	learn: 0.3039067	test: 0.3082041	best: 0.3082041 (199)	total: 13.8s	remaining: 54.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3080593334
bestIteration = 241

Shrink model to first 242 iterations.

🌀 Fold 2
0:	learn: 0.6679277	test: 0.6679846	best: 0.6679846 (0)	total: 64.1ms	remaining: 1m 4s
100:	learn: 0.3092756	test: 0.3101847	best: 0.3101847 (100)	total: 4.93s	remaining: 43.9s
200:	learn: 0.3043821	test: 0.3072015	best: 0.3072015 (200)	total: 7.69s	remaining: 30.6s
300:	learn: 0.3018495	test: 0.3072052	best: 0.3070608 (256)	total: 11.4s	remaining: 26.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3070607633
bestIteration = 256

Shrink model to first 257 iterations.

🌀 Fold 3
0:	learn: 0.6678844	test: 0.6679904	best: 0.6679904 (0)	total: 18.2ms

In [15]:
from google.colab import files
files.download("submission_catboost_thresh_opt.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
import pandas as pd
import numpy as np
from google.colab import files

# ✅ 예측 결과 불러오기
catboost = pd.read_csv("/content/submission_catboost_thresh_opt.csv")
stacking = pd.read_csv("/content/submission_stacking_lgb_meta.csv")
sample = pd.read_csv("/content/open_1/sample_submission.csv")

# ✅ ID 동일성 확인
assert all(catboost["ID"] == stacking["ID"])
assert all(sample["ID"] == stacking["ID"])

# ✅ 1. Soft Voting (가중치 조정 가능)
submission_vote = pd.DataFrame()
submission_vote["ID"] = sample["ID"]
submission_vote["Cancer"] = 0.6 * catboost["Cancer"] + 0.4 * stacking["Cancer"]

# ✅ float → int 변환 (이진 분류용)
submission_vote["Cancer"] = (submission_vote["Cancer"] > 0.5).astype(int)

# ✅ NaN 확인 및 저장
assert submission_vote.isnull().sum().sum() == 0
submission_vote.to_csv("/content/submission_weighted_cat_stacking_binary.csv", index=False)
print("✅ Soft Voting 결과 저장 완료")

# ✅ 2. Meta 앙상블 (평균)
submission_meta = pd.DataFrame()
submission_meta["ID"] = sample["ID"]
submission_meta["Cancer"] = ((catboost["Cancer"] + stacking["Cancer"]) / 2 > 0.5).astype(int)

# ✅ NaN 확인 및 저장
assert submission_meta.isnull().sum().sum() == 0
submission_meta.to_csv("/content/submission_meta_combined_avg_binary.csv", index=False)
print("✅ Meta 평균 결과 저장 완료")

# ✅ 파일 다운로드 (Colab 전용)
files.download("/content/submission_weighted_cat_stacking_binary.csv")
files.download("/content/submission_meta_combined_avg_binary.csv")


✅ Soft Voting 결과 저장 완료
✅ Meta 평균 결과 저장 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# 🔸 선택 시 다운로드 코드
# from google.colab import files
# files.download("submission_weighted_cat_stacking_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

# 데이터 로딩
train = pd.read_csv("/content/open_1/train.csv")
X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]

# 전처리
from sklearn.preprocessing import LabelEncoder
for col in X.columns:
    if X[col].dtype == "object":
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])

# 모델 준비
cat_model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6, verbose=0, random_seed=42)
stack_model = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)

# OOF 저장
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_cat = np.zeros(len(X))
oof_stack = np.zeros(len(X))

for train_idx, val_idx in cv.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train = y.iloc[train_idx]

    cat_model.fit(X_train, y_train)
    oof_cat[val_idx] = cat_model.predict_proba(X_val)[:, 1]

    stack_model.fit(X_train, y_train)
    oof_stack[val_idx] = stack_model.predict_proba(X_val)[:, 1]

# 저장
pd.DataFrame({"ID": train["ID"], "Cancer": oof_cat}).to_csv("/content/oof_catboost.csv", index=False)
pd.DataFrame({"ID": train["ID"], "Cancer": oof_stack}).to_csv("/content/oof_stacking.csv", index=False)
print("✅ OOF 예측 저장 완료")


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016953 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1123
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

In [25]:
# ✅ 라이브러리
import pandas as pd
import numpy as np
import optuna
from sklearn.metrics import log_loss, f1_score
from sklearn.model_selection import StratifiedKFold

# ✅ 데이터 로딩 (OOF 기반 튜닝을 위한 전제)
# CatBoost, Stacking 모델의 OOF 예측값 필요
oof_cat = pd.read_csv("/content/oof_catboost.csv")   # ID, Cancer
oof_stack = pd.read_csv("/content/oof_stacking.csv") # ID, Cancer
train = pd.read_csv("/content/open_1/train.csv")      # ID, Cancer 포함

y_true = train["Cancer"].values

# ✅ ID 정렬 확인
assert all(oof_cat["ID"] == oof_stack["ID"])
assert all(oof_cat["ID"] == train["ID"])

# ✅ Optuna 목적 함수 정의
def objective(trial):
    w_cat = trial.suggest_float("w_cat", 0.0, 1.0)
    w_stack = 1.0 - w_cat

    blended = w_cat * oof_cat["Cancer"] + w_stack * oof_stack["Cancer"]
    pred_binary = (blended > 0.5).astype(int)

    # 선택 기준: F1 score or LogLoss
    return 1.0 - f1_score(y_true, pred_binary)  # maximize F1 → minimize 1-F1

# ✅ Optuna 실험 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# ✅ 최적 가중치 출력
best_w_cat = study.best_params["w_cat"]
best_w_stack = 1.0 - best_w_cat
print(f"✅ 최적 가중치: CatBoost = {best_w_cat:.4f}, Stacking = {best_w_stack:.4f}")


[I 2025-06-19 06:19:08,754] A new study created in memory with name: no-name-38248ff6-12b9-430f-999a-21be01f6eed5
[I 2025-06-19 06:19:08,769] Trial 0 finished with value: 0.6906180941538873 and parameters: {'w_cat': 0.2367778115834417}. Best is trial 0 with value: 0.6906180941538873.
[I 2025-06-19 06:19:08,784] Trial 1 finished with value: 0.7162610619469026 and parameters: {'w_cat': 0.6226204999475137}. Best is trial 0 with value: 0.6906180941538873.
[I 2025-06-19 06:19:08,796] Trial 2 finished with value: 0.7315569413932392 and parameters: {'w_cat': 0.9490465517460254}. Best is trial 0 with value: 0.6906180941538873.
[I 2025-06-19 06:19:08,807] Trial 3 finished with value: 0.6860759493670886 and parameters: {'w_cat': 0.15615144617880616}. Best is trial 3 with value: 0.6860759493670886.
[I 2025-06-19 06:19:08,818] Trial 4 finished with value: 0.7316035729476819 and parameters: {'w_cat': 0.9543876659078594}. Best is trial 3 with value: 0.6860759493670886.
[I 2025-06-19 06:19:08,830] Tr

✅ 최적 가중치: CatBoost = 0.0087, Stacking = 0.9913


In [26]:
# ✅ 테스트셋 예측 결과 로딩
test_cat = pd.read_csv("/content/submission_catboost_thresh_opt.csv")
test_stack = pd.read_csv("/content/submission_stacking_lgb_meta.csv")
sample = pd.read_csv("/content/open_1/sample_submission.csv")

# ✅ 예측 병합 및 Threshold 적용
final_pred = (
    best_w_cat * test_cat["Cancer"] + best_w_stack * test_stack["Cancer"]
)
submission = sample.copy()
submission["Cancer"] = (final_pred > 0.5).astype(int)

# ✅ 저장
submission.to_csv("/content/submission_optuna_weighted_binary.csv", index=False)
print("✅ 제출 파일 저장 완료 → submission_optuna_weighted_binary.csv")


✅ 제출 파일 저장 완료 → submission_optuna_weighted_binary.csv


In [27]:

from google.colab import files
files.download("submission_optuna_weighted_binary.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
import pandas as pd
import numpy as np
import shap
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 데이터 로딩
train = pd.read_csv("/content/open_1/train.csv")
test = pd.read_csv("/content/open_1/test.csv")
submission = pd.read_csv("/content/open_1/sample_submission.csv")

X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ LightGBM 학습 후 SHAP 중요도 계산
model = LGBMClassifier(n_estimators=200, learning_rate=0.05, random_state=42)
model.fit(X, y)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)[1]  # class 1 기준 SHAP

# ✅ Top 10 중요한 feature 선택
shap_importance = np.abs(shap_values).mean(axis=0)
top10_idx = np.argsort(shap_importance)[-10:]
top10_features = X.columns[top10_idx].tolist()

# ✅ 파생 변수 생성 (곱, 나눗셈)
for i in range(len(top10_features)):
    for j in range(i + 1, len(top10_features)):
        f1, f2 = top10_features[i], top10_features[j]
        X[f"{f1}_x_{f2}"] = X[f1] * X[f2]
        X_test[f"{f1}_x_{f2}"] = X_test[f1] * X_test[f2]
        X[f"{f1}_div_{f2}"] = X[f1] / (X[f2] + 1e-5)
        X_test[f"{f1}_div_{f2}"] = X_test[f1] / (X_test[f2] + 1e-5)

# ✅ 스태킹 모델 정의
base_models = [
    ("lgbm", LGBMClassifier(random_state=42)),
    ("xgb", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ("cat", CatBoostClassifier(verbose=0, random_state=42)),
]
meta_model = LGBMClassifier(random_state=42)

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1
)

# ✅ Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("/content/submission_shap_interaction_stacking.csv", index=False)
print("✅ SHAP 기반 상호작용 + 스태킹 결과 저장 완료")


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.048703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1123
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439


/usr/local/lib/python3.11/dist-packages/shap/explainers/_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003915 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.25, Best F1 Score: 0.48605
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
✅ SHAP 기반 상호작용 + 스태킹 결과 저장 완료


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [31]:

from google.colab import files
files.download("submission_shap_interaction_stacking.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 데이터 로딩
train = pd.read_csv("/content/open_1/train.csv")
test = pd.read_csv("/content/open_1/test.csv")
submission = pd.read_csv("/content/open_1/sample_submission.csv")

X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 중요도 기준 Top 5 변수 추출 (LightGBM 사용)
lgb_temp = LGBMClassifier(n_estimators=200, learning_rate=0.05, random_state=42)
lgb_temp.fit(X, y)

importances = pd.Series(lgb_temp.feature_importances_, index=X.columns)
top5_features = importances.sort_values(ascending=False).head(5).index.tolist()

# ✅ 곱(product) 기반 상호작용 변수 생성
for i in range(len(top5_features)):
    for j in range(i + 1, len(top5_features)):
        f1, f2 = top5_features[i], top5_features[j]
        X[f"{f1}_x_{f2}"] = X[f1] * X[f2]
        X_test[f"{f1}_x_{f2}"] = X_test[f1] * X_test[f2]

# ✅ 스태킹 모델 정의
base_models = [
    ("lgbm", LGBMClassifier(random_state=42)),
    ("xgb", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ("cat", CatBoostClassifier(verbose=0, random_state=42)),
]
meta_model = LGBMClassifier(random_state=42)

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1
)

# ✅ F1 최적 threshold 찾기
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("/content/submission_importance_top5_product_stacking.csv", index=False)
print("✅ Top5 변수 곱만 조합 + Stacking 결과 저장 완료")


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045108 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1123
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003841 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[Ligh

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001726 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463


...

Best Threshold: 0.33, Best F1 Score: 0.48571


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002852 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Top5 변수 곱만 조합 + Stacking 결과 저장 완료


In [33]:
from google.colab import files
files.download("submission_importance_top5_product_stacking.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier

# ✅ 데이터 로딩
train = pd.read_csv("/content/open_1/train.csv")
test = pd.read_csv("/content/open_1/test.csv")
submission = pd.read_csv("/content/open_1/sample_submission.csv")

X = train.drop(columns=["ID", "Cancer"])
y = train["Cancer"]
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 스태킹 모델 정의 (Meta만 CatBoost)
base_models = [
    ("lgbm", LGBMClassifier(random_state=42)),
    ("xgb", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ("cat", CatBoostClassifier(verbose=0, random_state=42)),
]

meta_model = CatBoostClassifier(verbose=0, random_state=42)

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method="predict_proba",
    cv=5,
    n_jobs=-1
)

# ✅ Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("/content/submission_stacking_meta_catboost.csv", index=False)
print("✅ CatBoost 메타 모델 기반 stacking 결과 저장 완료")


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Threshold: 0.24, Best F1 Score: 0.48419
✅ CatBoost 메타 모델 기반 stacking 결과 저장 완료


In [35]:
from google.colab import files
files.download("submission_stacking_meta_catboost.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
import pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import RidgeClassifierCV, LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
import optuna
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# — 데이터 로딩 & 전처리
def preprocess(df, enc=None):
    df = df.copy(); enc = enc or {}
    for col in df:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in enc:
                enc[col] = LabelEncoder().fit(df[col])
            df[col] = enc[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, enc

train = pd.read_csv("/content/open_1/train.csv")
test = pd.read_csv("/content/open_1/test.csv")
sub = pd.read_csv("/content/open_1/sample_submission.csv")
X, enc = preprocess(train.drop(["ID","Cancer"], axis=1))
y = train["Cancer"]
X_test, _ = preprocess(test.drop("ID", axis=1), enc)

# — CAST pseudo-labeling 시작
base = CatBoostClassifier(n_estimators=300, learning_rate=0.05, random_seed=42, verbose=0)
base.fit(X,y)
probs = base.predict_proba(X_test)[:,1]

# 밀도 기반 확신도 조정
kmeans = KMeans(n_clusters=5, random_state=42).fit(X)
dens = np.zeros(len(X_test))
for i,cluster in enumerate(kmeans.predict(X_test)):
    dens[i] = (kmeans.labels_ == cluster).mean()
conf = probs * dens

mask = (conf>0.9) | (conf<0.1)
pseudo_X = X_test[mask]
pseudo_y = (probs[mask] > 0.5).astype(int)
X_aug = pd.concat([X, pseudo_X], ignore_index=True)
y_aug = np.concatenate([y, pseudo_y], axis=0)

# — stacking input 구성
models = [
    ("lgb", LGBMClassifier(random_state=42)),
    ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
    ("cat", CatBoostClassifier(verbose=0, random_state=42))
]
kf = StratifiedKFold(5, shuffle=True, random_state=42)
oof = np.zeros((len(X_aug), len(models)))
oof_test = np.zeros((len(X_test), len(models)))

for i,(_,m) in enumerate(models):
    temp_oof = cross_val_predict(m, X_aug, y_aug, cv=kf, method="predict_proba")[:,1]
    m.fit(X_aug, y_aug)
    temp_test = m.predict_proba(X_test)[:,1]
    oof[:,i] = temp_oof
    oof_test[:,i] = temp_test

stack_X = oof
stack_test = oof_test

# — meta learner 탐색(optuna)
def objective(trial):
    ridge_alpha = trial.suggest_loguniform("ridge_alpha", 1e-3, 10)
    clf = RidgeClassifierCV(alphas=[ridge_alpha], cv=kf)
    scores = cross_val_predict(clf, stack_X, y_aug, cv=kf, method="predict")
    return 1 - f1_score(y_aug, scores)

opt = optuna.create_study(direction="minimize")
opt.optimize(objective, n_trials=25)
alpha = opt.best_params["ridge_alpha"]
meta = RidgeClassifierCV(alphas=[alpha], cv=kf)
meta.fit(stack_X, y_aug)

# — 예측 후 submission
pred_final = meta.predict(stack_test)
sub["Cancer"] = pred_final
sub.to_csv("/content/submission_cast_enhanced_stacking.csv", index=False)
print("✅ 완성: CAST + Optuna stacking 결과 제출!")


[LightGBM] [Info] Number of positive: 8824, number of negative: 95650
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021291 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1124
[LightGBM] [Info] Number of data points in the train set: 104474, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.084461 -> initscore=-2.383220
[LightGBM] [Info] Start training from score -2.383220
[LightGBM] [Info] Number of positive: 8824, number of negative: 95650
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1123
[LightGBM] [Info] Number of data points in the train set: 104474, number of used features: 14
[LightGBM] [Info] 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:51:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

✅ 완성: CAST + Optuna stacking 결과 제출!


In [39]:
from google.colab import files
files.download("submission_cast_enhanced_stacking.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
# ===============================================================
# 1) 라이브러리
# ===============================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ---------------------------------------------------------------
# 2) 경로 함수
# ---------------------------------------------------------------
def p(fn):
    return f"/content/open_1/{fn}"

# ---------------------------------------------------------------
# 3) 데이터 로딩
# ---------------------------------------------------------------
train = pd.read_csv(p("train.csv"))
test  = pd.read_csv(p("test.csv"))
sub   = pd.read_csv(p("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ---------------------------------------------------------------
# 4) 전처리
#    - 수치형 : median 대체 + IQR-clip
#    - 범주형 : mode 대체, LabelEncode
# ---------------------------------------------------------------
def iqr_clip(s, q1=0.01, q2=0.99):
    lo, hi = s.quantile(q1), s.quantile(q2)
    return s.clip(lo, hi)

def preprocess(df, enc_dict=None):
    df = df.copy()
    enc_dict = enc_dict or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in enc_dict:
                enc_dict[col] = LabelEncoder().fit(df[col])
            df[col] = enc_dict[col].transform(df[col])
            df[col] = SimpleImputer(strategy="most_frequent").fit_transform(df[[col]])
        else:
            df[col] = SimpleImputer(strategy="median").fit_transform(df[[col]])
            df[col] = iqr_clip(df[col])
    return df, enc_dict

X,  enc = preprocess(X)
X_test, _ = preprocess(X_test, enc)

# ---------------------------------------------------------------
# 5) 베이스 모델
# ---------------------------------------------------------------
base_models = [
    ("lgb", LGBMClassifier(
        n_estimators=800, learning_rate=0.05,
        max_depth=-1, subsample=0.8, colsample_bytree=0.8,
        random_state=42
    )),
    ("xgb", XGBClassifier(
        n_estimators=700, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, use_label_encoder=False, eval_metric="logloss"
    )),
    ("cat", CatBoostClassifier(
        iterations=700, learning_rate=0.05, depth=6,
        l2_leaf_reg=3, random_seed=42, verbose=0
    ))
]

# ---------------------------------------------------------------
# 6) 메타 모델 (LGBM – 살짝 더 깊게 + 규제)
# ---------------------------------------------------------------
meta_model = LGBMClassifier(
    n_estimators=600, learning_rate=0.05,
    max_depth=4, num_leaves=31,
    colsample_bytree=0.8, reg_lambda=1.0,
    random_state=42
)

# ---------------------------------------------------------------
# 7) StackingClassifier
# ---------------------------------------------------------------
stack = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method="predict_proba",
    passthrough=True,                        # ← 원본 feature도 meta에 전달
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1
)

# ---------------------------------------------------------------
# 8) OOF 예측 & Threshold 탐색 (PR-curve 기반 F1 최대)
# ---------------------------------------------------------------
oof = cross_val_predict(
    stack, X, y, cv=StratifiedKFold(5, shuffle=True, random_state=42),
    method="predict_proba", n_jobs=-1
)[:, 1]

prec, rec, thr = precision_recall_curve(y, oof)
f1 = 2 * prec * rec / (prec + rec + 1e-8)
best_thr = thr[np.argmax(f1)]
print(f"Best Threshold = {best_thr:.4f} | F1(O.O.F.) = {f1.max():.5f}")

# ---------------------------------------------------------------
# 9) 최종 학습 & 예측
# ---------------------------------------------------------------
stack.fit(X, y)
test_probs = stack.predict_proba(X_test)[:, 1]
sub["Cancer"] = (test_probs > best_thr).astype(int)
sub.to_csv("/content/submission_stack_v2_lgbMeta.csv", index=False)
print("✅ submission_stack_v2_lgbMeta.csv 저장 완료")


Best Threshold = 0.1841 | F1(O.O.F.) = 0.48373
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023580 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1887
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ submission_stack_v2_lgbMeta.csv 저장 완료


In [41]:
from google.colab import files
files.download("submission_stack_v2_lgbMeta.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score, precision_recall_curve
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리
def preprocess(df, encoders=None, is_train=True):
    df = df.copy()
    encoders = encoders or {}
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns
    cat_cols = df.select_dtypes(include=["object"]).columns

    for col in num_cols:
        df[col] = SimpleImputer(strategy="median").fit_transform(df[[col]])

    for col in cat_cols:
        df[col] = df[col].astype(str)
        if is_train:
            encoders[col] = LabelEncoder().fit(df[col])
        df[col] = encoders[col].transform(df[col])

    df[num_cols] = RobustScaler().fit_transform(df[num_cols])

    return df, encoders

X, encoders = preprocess(X, is_train=True)
X_test, _ = preprocess(X_test, encoders, is_train=False)

# ✅ 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ 스태킹 모델
stack_model = StackingClassifier(
    estimators=[
        ('lgbm', model_lgbm),
        ('xgb', model_xgb),
        ('cat', model_cat)
    ],
    final_estimator=meta_model,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    stack_method="predict_proba",
    passthrough=True,
    n_jobs=-1
)

# ✅ OOF 기반 threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
prec, rec, thresholds = precision_recall_curve(y, oof_preds)
f1s = 2 * prec * rec / (prec + rec + 1e-6)
best_thresh = thresholds[np.argmax(f1s)]
print(f"📌 Best Threshold (PR-Curve): {best_thresh:.4f}, F1: {np.max(f1s):.5f}")

# ✅ 전체 학습 및 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("/content/submission_stacking_lgb_meta_refined.csv", index=False)

print("✅ refined stacking 결과 저장 완료")


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014563 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

...

[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011404 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


📌 Best Threshold (PR-Curve): 0.2093, F1: 0.48642
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ refined stacking 결과 저장 완료


In [43]:
from google.colab import files
files.download("submission_stacking_lgb_meta_refined.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt

# ✅ 데이터 경로
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, enc = preprocess(X)
X_test, _ = preprocess(X_test, enc)

# ✅ base models
model_lgb = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31,
                           min_child_samples=5, min_gain_to_split=0.001, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)

# ✅ meta model
meta_model = LGBMClassifier(learning_rate=0.03, max_depth=3, num_leaves=15, random_state=42)

# ✅ Stacking Classifier
stack_model = StackingClassifier(
    estimators=[("lgb", model_lgb), ("xgb", model_xgb), ("cat", model_cat)],
    final_estimator=meta_model,
    stack_method="predict_proba",
    passthrough=True,
    cv=5,
    n_jobs=-1
)

# ✅ OOF 예측값
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = cross_val_predict(stack_model, X, y, cv=kf, method="predict_proba")[:, 1]

# ✅ Threshold 탐색
thresholds = np.linspace(0.1, 0.9, 81)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold = {best_thresh:.4f} | F1(O.O.F.) = {max(f1s):.5f}")

# ✅ 재학습 + 확률 보정
stack_model.fit(X, y)
cal_model = CalibratedClassifierCV(stack_model, method='sigmoid', cv='prefit')
cal_model.fit(X, y)

# ✅ 예측 및 저장
test_preds = cal_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stack_improved.csv", index=False)

# ✅ 예측 분포 시각화
plt.hist(test_preds, bins=50, color='skyblue', edgecolor='black')
plt.title("Test Predict Probability Distribution")
plt.xlabel("Probability")
plt.ylabel("Frequency")
plt.grid(True)
plt.savefig("test_proba_distribution.png")
plt.show()


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1888
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1887
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from google.colab import files
files.download("submission_stack_improved.csv")
